In [1]:
import os
import csv
import pandas as pd

In [62]:
pwd = os.getcwd()
file_paths = ['/scraped_data/scrape_of_rows_odd.csv', '/scraped_data/scrape_of_rows_even.csv']

In [8]:
def read_to_list(index):
    with open(pwd + file_paths[index], 'r') as f:
        reader = csv.reader(f)
        return list(reader)

In [76]:
odd_rows_raw = read_to_list(0)
even_rows_raw = read_to_list(1)

# <p align=center> Pre Processing Data </p>
## <p>Noticing that the data from the website is displayed rather annoyingly, in the form of span rows, I have decided to alternate the rows with the view of pasting together both list into a DataFrame. To acheieve this, I have to pre process the data to have it in the correct sizing, format to import into the DataFrame.  The odd rows in particular, the way the data was displayed resulted in some entries being not of the same length, thus being out of sync with most of the data pulled from this subsection of rows.  The best way I came about to solving this issue, was to isolate the rows that are not needed, in particular the empty lists, and those that took the header of the table, and remove them.  Then insert a generic entry into the lists that remain, to align the data with the rest.  I will go through and clean this data once It is active in the DataFrame</p>

In [15]:
odd_df = pd.DataFrame(odd_rows_raw, columns=["id", "track", "team", "driver_num", "driver", "model", "engine", "tyre", "grid_pos", "fin_pos", "notes"]) 
print(odd_df.shape)

odd_df = odd_df.iloc[odd_df.index[46:68]].shift(periods=2, axis="columns")

(344, 11)


,id,track,team,driver_num,driver,model,engine,tyre,grid_pos,fin_pos,notes
0,n,Grand Prix,Team,N°,Driver,Model,Engine,Tyre,Grid,Race,None
1,None,None,None,None,None,None,None,None,None,None,None
2,167,Australia,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,11,ab,Engine
3,None,None,None,None,None,None,None,None,None,None,None
4,,,,,,,,,,,


In [28]:
# Cleaning the list removing unnessecary lists and adding new indexes where needed.  
for ls in odd_rows_raw: 
    if len(ls) == 0 or ls[0] == '' or ls[0] == 'n': 
        odd_rows_raw.remove(ls)
    if len(ls) <= 10 or len(ls) == 9: 
        ls.insert(0, 'id')
        ls.insert(1, 'track')
        print(ls)
    else: 
        pass

In [42]:
# Copy of the uniformed list as bacup. 
odd_rows_removed = [row.copy() for row in odd_rows_raw]

In [52]:
# Adding the year to the list throughout the season results. 
def insert_years(list_name):
    races_per_season = { 
        '2015' : '19',
        '2016' : '21',
        '20116': '21',
        '2017' : '20',
        '2018' : '21',
        '2019' : '21', 
        '2020' : '17', 
        '2021' : '16'
    }

    i = 0
    for k,v in races_per_season.items(): 
        total_races = 0
        while total_races < int(v):
            list_name[i].insert(1, k)
            total_races += 1
            i += 1

# <p align=center>Creating the first DataFrame</p>
## Now that we have pre processed our odd data, by adding the requried indices to make it conform to the main list we can create our first DataFrame.

In [64]:
odd_df = pd.DataFrame(odd_rows_removed, columns=["id", "year", "track", "team", "driver_num", "driver", "model", "engine", "tyre", "grid_pos", "fin_pos", "notes"])
odd_df.head()

,id,year,track,team,driver_num,driver,model,engine,tyre,grid_pos,fin_pos,notes
0,167,2015,Australia,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,11,ab,Engine
1,168,2015,Malaysia,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,6,7,
2,169,2015,China,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,13,17,Engine
3,170,2015,Bahrain,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,15,ab,Electrics
4,171,2015,Spain,Scuderia Toro Rosso,33,VERSTAPPEN Max,STR10,Renault,Pirelli,6,11,


# <p align=center> Now for the even rows </p>
## Now we repeat the above process with the even rows

In [82]:
for ls in even_rows_raw: 
    if len(ls) == 0: 
        even_rows_raw.remove(ls)
    elif len(ls) == 9: 
        ls.insert(0, 'id')
        ls.insert(1, 'track')
    else:
        pass

In [85]:
even_rows_removed = [row.copy() for row in even_rows_raw]

In [88]:
even_rows_removed[0]

['id',
 '2015',
 'track',
 'Scuderia Toro Rosso',
 '55',
 'SAINZ Carlos',
 'STR10',
 'Renault',
 'Pirelli',
 '7',
 '9',
 '']